In [1]:
%load_ext watermark
%watermark -u -d -p numpy,matplotlib,tensorflow,sklearn

Last updated: 2021-06-10

numpy     : 1.19.5
matplotlib: 3.3.4
tensorflow: 2.4.1
sklearn   : 0.24.2



In [2]:
import os
import cv2
import time
import numpy as np
import imutils

from imutils.video import VideoStream
from tensorflow.keras.models import load_model

In [3]:
# 학습된 모델을 읽어 온다
EmptyNet = load_model("empty_v2_K-Fold.model")

In [4]:
# 스트리밍 영상을 받아 분류를 하기 위한 데이터로 변환한다 (Gray 변환 및 Resizing 처리)
# 이미 학습되어 있는 예측함수로 선반이 비었는지, 재고가 있는지를 예측하고 확율을 반환한다

def detect_and_predict_Empty(frame, EmptyNet):
    
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # img = cv2.cvtColor(frame, cv2.IMREAD_GRAYSCALE)
    new_img = cv2.resize(img,(70,70))
    ex = np.array(new_img).reshape(-1,70,70,1)
    ex = ex / 255.0
    
    pred = EmptyNet.predict(ex)
    
    return pred

In [16]:
# video stream을 초기화하고 camera sensor가 warm up할 수 있도록 기다려준다
vs = VideoStream(src=0).start()
time.sleep(2.0)

In [17]:
# video stream을 받아 예측 결과를 기반으로 선반이 비어있는지, 재고가 있는지 모니터링 한다

while True:
    # 비디오 스트리밍을 읽어 드리고, 화면에 표시할 크기(여기선 400 pixel)만큼 조정해 준다
    frame = vs.read()
    frame = cv2.flip(frame,1,1) # 거울처럼 영상이 보이도록 화면을 flip해 준다
    frame = imutils.resize(frame, width=400)

    # 예측 함수를 호출한다
    preds = detect_and_predict_Empty(frame, EmptyNet)
    
    # 예측 결과 확율을 기반으로 선반이 비어있는지 재고가 있는지 구분한다 (여기선 50%를 기준으로 삼았다)
    label = "Non-Empty" if preds > 0.5 else "Empty"
    color = (0, 0, 255) if label == "Empty" else (0, 255, 0)
    
    # 확율과 label을 화면에 표기해 준다
    # label = f"{label}: {preds * 100}%"
    label = "{}: {:.2f}%".format(label, float(preds) * 100)
    cv2.putText(frame, label, (200, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # 스트리밍 영상을 실제로 화면에 보여준다
    cv2.imshow("Frame", frame)
    
    key = cv2.waitKey(1) & 0xFF
    # q키를 누르면 while loop를 빠져나간다
    if key == ord("q"):
        break

cv2.destroyAllWindows()
vs.stop()